In [1]:
# skorch : pytorch를 쉽게 구현할 수 있는 라이브러리

from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train= X_train.reshape(-1,784)
X_test= X_test.reshape(-1,784)
X_train=X_train/255.
X_test=X_test/255.

In [2]:
import torch
X_train=torch.from_numpy(X_train).float()
y_train=torch.from_numpy(y_train.astype('int32')).long()
X_test=torch.from_numpy(X_test).float()
y_test=torch.from_numpy(y_test.astype('int32')).long()
print(X_train.shape)
print(X_test.shape)
X_train=X_train.cuda()
y_train=y_train.cuda()
X_test=X_test.cuda()
y_test=y_test.cuda()

torch.Size([60000, 784])
torch.Size([10000, 784])


In [3]:
import torch.nn as nn
import torch.nn.functional as F
# 신경망 구성
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=-1)
model=Net().cuda()

In [4]:
from skorch import NeuralNetClassifier
net = NeuralNetClassifier(Net,max_epochs=20,lr=0.1)
net.fit(X_train, y_train)

import numpy as np
# 모형의 정확도 계산
pred = net.predict(X_test)
y_test = y_test.cpu().numpy()
accuracy = np.mean(pred == y_test)
accuracy


  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6352       0.9007        0.3482  2.5205
      2        0.3135       0.9197        0.2852  1.7794
      3        0.2629       0.9299        0.2462  1.7181
      4        0.2273       0.9386        0.2175  1.8557
      5        0.1997       0.9437        0.1952  1.6770
      6        0.1776       0.9487        0.1771  1.6776
      7        0.1596       0.9527        0.1627  1.9128
      8        0.1449       0.9571        0.1515  1.7754
      9        0.1326       0.9599        0.1423  1.6364
     10        0.1222       0.9618        0.1349  1.7440
     11        0.1133       0.9642        0.1286  1.7436
     12        0.1055       0.9653        0.1235  1.6532
     13        0.0986       0.9662        0.1189  1.6407
     14        0.0926       0.9672        0.1151  1.6461
     15        0.0871       0.9677        0.1115  1.6419
     16        0.0821       0.9

0.9731

In [5]:
from torchinfo import summary
summary(model, input_size=(100,784))


Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [100, 10]                 --
├─Linear: 1-1                            [100, 128]                100,480
├─Linear: 1-2                            [100, 10]                 1,290
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
Total mult-adds (M): 10.18
Input size (MB): 0.31
Forward/backward pass size (MB): 0.11
Params size (MB): 0.41
Estimated Total Size (MB): 0.83